In [1]:
from dotenv import dotenv_values
from requests import get
from requests.models import HTTPError
from bs4 import BeautifulSoup

import pandas as pd
import random

In [2]:
CONFIG = dotenv_values('../.env')

In [3]:
def requisicao_api_alura():
    try:
        response = get('https://cursos.alura.com.br/api/categorias')
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'Um erro HTTP ocorreu: {http_err}')
    except Exception as err:
        print(f'Um erro ocorreu: {err}')
    else:
        return response.json()

In [8]:
dados = requisicao_api_alura()
urls_categorias = {}
url = 'https://cursos.alura.com.br/forum/categoria-'
for item in dados:
    urls_categorias[item['nome']] = f'{url}{item["slug"]}/todos'
urls_categorias

{'Programação': 'https://cursos.alura.com.br/forum/categoria-programacao/todos',
 'Front-end': 'https://cursos.alura.com.br/forum/categoria-front-end/todos',
 'Data Science': 'https://cursos.alura.com.br/forum/categoria-data-science/todos',
 'DevOps': 'https://cursos.alura.com.br/forum/categoria-devops/todos',
 'UX & Design': 'https://cursos.alura.com.br/forum/categoria-design-ux/todos',
 'Mobile': 'https://cursos.alura.com.br/forum/categoria-mobile/todos',
 'Inovação & Gestão': 'https://cursos.alura.com.br/forum/categoria-inovacao-gestao/todos'}

In [3]:
def requisicao(url):
    cookies = {'caelum.login.token':CONFIG['COOKIE']}
    try:
        response = get(url, cookies=cookies)
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'Um erro HTTP ocorreu: {http_err}')
    except Exception as err:
        print(f'Um erro ocorreu: {err}')
    else:
        soup = BeautifulSoup(response.text,'html.parser')
        return soup

In [20]:
paginas_categorias = {}
for chave, valor in urls_categorias.items():
    soup = requisicao(valor)
    ultima_pagina = int(soup.find('nav', {'class': 'busca-paginacao-links'}).get_text().split()[-1])
    paginas_categorias[chave] = random.sample(range(1, ultima_pagina+1), 250)
paginas_categorias

{'Programação': [2562,
  2761,
  3426,
  898,
  329,
  2780,
  2837,
  802,
  2093,
  3605,
  3723,
  2859,
  852,
  247,
  3041,
  2133,
  2125,
  680,
  804,
  2267,
  2414,
  3171,
  2432,
  3319,
  294,
  3117,
  1237,
  1282,
  2408,
  462,
  2554,
  2039,
  2479,
  3253,
  465,
  3393,
  2457,
  1326,
  668,
  419,
  223,
  10,
  2117,
  1323,
  1382,
  3395,
  1223,
  335,
  3264,
  2681,
  2906,
  3055,
  2290,
  3766,
  2721,
  1020,
  3843,
  1558,
  3372,
  256,
  2241,
  3778,
  1904,
  1884,
  193,
  3638,
  2814,
  2237,
  2051,
  1169,
  656,
  3184,
  2739,
  3564,
  3631,
  2698,
  600,
  2539,
  2059,
  1386,
  2735,
  886,
  3460,
  1304,
  1895,
  2067,
  1868,
  3497,
  3609,
  2809,
  1749,
  3838,
  586,
  2470,
  518,
  3265,
  1379,
  448,
  3344,
  131,
  2672,
  2370,
  2284,
  2718,
  3118,
  115,
  2987,
  3819,
  3493,
  2243,
  2883,
  2417,
  1236,
  2931,
  954,
  858,
  3448,
  3075,
  1645,
  2072,
  466,
  343,
  3148,
  1221,
  2706,
  1586,
  933,


In [47]:
def dataframe_links_categoria(categoria):
    titulos = []
    links = []
    for pagina in paginas_categorias[categoria]:
        soup  = requisicao(f'{urls_categorias[categoria]}/{pagina}')
        itens  = soup.find_all('a', {'class': 'forumList-item-subject-info-title-link'})
        for item in itens:
            titulos.append(item.text.strip().split('\n')[0])
            links.append(f'https://cursos.alura.com.br{item["href"]}')

    dados = pd.DataFrame({'titulo': titulos, 'links': links})
    dados.to_csv(f'../dados/{categoria}.csv')

In [46]:
dataframe_links_categoria('Data Science')

In [48]:
dataframe_links_categoria('Programação')

In [49]:
dataframe_links_categoria('Inovação & Gestão')

In [50]:
dataframe_links_categoria('Front-end')

In [57]:
dataframe_links_categoria('DevOps')

In [58]:
dataframe_links_categoria('UX & Design')

In [59]:
dataframe_links_categoria('Mobile')

In [5]:
data_science = pd.read_csv('../dados/Data Science.csv')

In [9]:
textos = []
i = 0
for url in data_science['links']:
    i += 1
    print(i)
    soup = requisicao(url)
    textos.append(soup.find('div', {'class': 'topic-post-question-content'}).find('section', {'class': 'topic-post-content'}).text.strip())
data_science['texto'] = textos
data_science.to_csv('../dados/Data Science com textos.csv')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [10]:
programacao = pd.read_csv('../dados/Programação.csv')

In [11]:
textos = []
for url in programacao['links']:
    soup = requisicao(url)
    textos.append(soup.find('div', {'class': 'topic-post-question-content'}).find('section', {'class': 'topic-post-content'}).text.strip())
programacao['texto'] = textos
programacao.to_csv('../dados/Programação com textos.csv')

In [14]:
devops = pd.read_csv('../dados/DevOps.csv')

In [16]:
textos = []
for url in devops['links']:
    soup = requisicao(url)
    textos.append(soup.find('div', {'class': 'topic-post-question-content'}).find('section', {'class': 'topic-post-content'}).text.strip())
devops['texto'] = textos
devops.to_csv('../dados/DevOps com textos.csv')

In [17]:
front_end = pd.read_csv('../dados/Front-end.csv')

In [19]:
textos = []
for url in front_end['links']:
    soup = requisicao(url)
    textos.append(soup.find('div', {'class': 'topic-post-question-content'}).find('section', {'class': 'topic-post-content'}).text.strip())
front_end['texto'] = textos
front_end.to_csv('../dados/Front-end com textos.csv')

In [20]:
mobile = pd.read_csv('../dados/Mobile.csv')

In [22]:
textos = []
for url in mobile['links']:
    soup = requisicao(url)
    textos.append(soup.find('div', {'class': 'topic-post-question-content'}).find('section', {'class': 'topic-post-content'}).text.strip())
mobile['texto'] = textos
mobile.to_csv('../dados/Mobile com textos.csv')

Um erro HTTP ocorreu: 500 Server Error:  for url: https://cursos.alura.com.br/forum/topico-realizar-o-curso-atraves-da-ide-do-android-studio-18311


AttributeError: 'NoneType' object has no attribute 'find'

In [24]:
textos

['A atualização ocorreu com sucesso, porém o seguinte erro apareceu logo após a mesma no cadastro.ts referente ao alerta, fazendo com que parasse de funcionar tanto no --lab quanto na geração do apk: \nC:\\Estudos\\Cursos\\ionic alura\\Ionic 2 # 1\\projeto\\aluraCar>ionic serve --lab\n\n> ionic-hello-world@ ionic:serve C:\\Estudos\\Cursos\\ionic alura\\Ionic 2 # 1\\projeto\\aluraCar\n> ionic-app-scripts serve "--v2" "--lab" "--address" "0.0.0.0" "--port" "8100" "--livereload-port" "35729"\n\n[23:59:08]  ionic-app-scripts 1.3.7\n[23:59:08]  watch started ...\n[23:59:08]  build dev started ...\n[23:59:08]  clean started ...\n[23:59:08]  clean finished in 4 ms\n[23:59:08]  copy started ...\n[23:59:08]  transpile started ...\n[23:59:24]  typescript: ...s/Cursos/ionic alura/Ionic 2 # 1/projeto/aluraCar/src/pages/cadastro/cadastro.ts, line: 37\n            Argument of type \'{ title: string; buttons: { text: string; handler: () => Promise<any>; }[]; }\' is not\n            assignable to para

In [26]:
mobile[mobile['links'] == 'https://cursos.alura.com.br/forum/topico-realizar-o-curso-atraves-da-ide-do-android-studio-18311']

,Unnamed: 0,titulo,links
3720,3720,Realizar o Curso através da IDE do Android-studio,https://cursos.alura.com.br/forum/topico-reali...


In [30]:
mobile.drop(3720, axis = 0, inplace = True)

In [36]:
for url in mobile[3720:]['links']:
    soup = requisicao(url)
    textos.append(soup.find('div', {'class': 'topic-post-question-content'}).find('section', {'class': 'topic-post-content'}).text.strip())
mobile['texto'] = textos
mobile.to_csv('../dados/Mobile com textos.csv')

In [4]:
ux_design = pd.read_csv('../dados/UX & Design.csv')

In [5]:
textos = []
for url in ux_design['links']:
    soup = requisicao(url)
    textos.append(soup.find('div', {'class': 'topic-post-question-content'}).find('section', {'class': 'topic-post-content'}).text.strip())
ux_design['texto'] = textos
ux_design.to_csv('../dados/UX & Design com textos.csv')

In [6]:
inovacao = pd.read_csv('../dados/Inovação & Gestão.csv')

In [7]:
textos = []
for url in inovacao['links']:
    soup = requisicao(url)
    textos.append(soup.find('div', {'class': 'topic-post-question-content'}).find('section', {'class': 'topic-post-content'}).text.strip())
inovacao['texto'] = textos
inovacao.to_csv('../dados/Inovação e Gestão com textos.csv')